In [1]:
# -*- coding: utf-8 -*-
"""diplom_test2.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1lCMapKzqQymHpHvYICnWnXUW3RzYBt06
"""

import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras

import os
import shutil
import time
from matplotlib import pyplot as plt 
import math

class deepRitz():

    def __init__(self, spatial_range, num_hidden=20, batch_size=200, num_iters=1000, lr_rate=1e-3, output_path='deepRitz'):
        self.spatial_range = spatial_range # for example, spatial range = [-1,1]
        self.num_hidden = num_hidden
        self.batch_size = batch_size #количество точек
        self.num_iters = num_iters
        self.lr_rate = lr_rate 
        self.output_path = output_path
        self.loss_history = []
        tf.reset_default_graph() 
        
    def build_nn(self, input):
        """fcnet for all pde_type"""
        num_hidden = self.num_hidden #количество нейронов внутри скрытых слоев (на одном слое)
        with tf.variable_scope("build_nn", reuse=tf.AUTO_REUSE): #создание переменных, повторное использование
            x = tf.layers.Dense(num_hidden, activation=tf.nn.tanh)(input) #добавление слоя, гиперболический тангенс (-1,1)
            x = tf.layers.Dense(num_hidden, activation=tf.nn.tanh)(x)
            x = x + tf.layers.Dense(num_hidden, activation=tf.nn.tanh)(x) #метод, который позволяет лучше найти коэффициенты
            x = tf.layers.Dense(num_hidden, activation=tf.nn.tanh)(x)
            output = tf.layers.Dense(1)(x)
        return output

    def alfa(self,x):
        a=self.spatial_range[0]
        b=self.spatial_range[1]
        return (x-a)*(b-x) #(x+1)(1-x)
      
    
    def beta(self,x):
        return 0

    def train(self):
        """train step"""
        
        z = tf.placeholder(tf.float32, shape=[None,1]) #передаем n точек за раз [100,1]

        u = self.alfa(z) * self.build_nn(z) + self.beta(z) #alpha(x)Net(x, theta) + beta(x)
        
        gradients = tf.gradients(u, z) #значение производной в соответствующей точке

        slopes = tf.square(gradients) #возведение в квадрат
            
        def rightside(z): #f 
            return abs(z)/ z

        loss = tf.reduce_mean(0.5*slopes-rightside(z)*u) #среднее значение (см пдф I(u))

        
        train_op = tf.train.AdamOptimizer(self.lr_rate).minimize(loss) #Параметры обучающей сети оптимизатора Адама
        # train_op = tf.compat.v1.train.optimizers.SGD(self.lr_rate).minimize(loss)
        
        if os.path.exists(self.output_path):
            shutil.rmtree(self.output_path)
        os.mkdir(self.output_path)
        
        saver = tf.train.Saver()
        with tf.Session() as sess: #начав сессию вам необходимо ее завершить или использовать блок with… as, например вот так
            sess.run(tf.global_variables_initializer()) #механизм для инициализации всех переменных за раз
            for iter in range(self.num_iters):
                #На каждом шаге “обучения” (оптимизации) эти точки выбираются заново (np.random.rand(self.batch_size, 1))
                batch_data = self.spatial_range[0] + (self.spatial_range[1]-self.spatial_range[0])*np.random.rand(self.batch_size, 1)
                loss_cur, _ = sess.run([loss, train_op], feed_dict={z: batch_data}) #feed_dict - передача данных
                self.loss_history.append(loss_cur)
                if (iter+1) % 100 == 0:
                # if (iter+1) % 10 == 0:
                    print('iteration: {}, loss: {:.4}'. format(iter+1, loss_cur))
            saver.save(sess, os.path.join(self.output_path, "model"), global_step=iter)
   
    def test(self, x, branch):
        checkpoint = tf.train.latest_checkpoint(self.output_path)
        saver = tf.train.Saver()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver.restore(sess, checkpoint)
            u = self.alfa(x) * self.build_nn(x) + self.beta(x) #приближенное решение
            if branch==1:
               u_test = sess.run(u)
            else: #считаем вторую производную
               gradients = tf.gradients(u, [x])[0] 
               ux=tf.reshape(gradients[:,0],[-1,1])
               u_test = sess.run(ux)
        return u_test #сеточная функция
    
    def numtest(self, num_points, branch=1):
        xs = np.linspace(self.spatial_range[0],self.spatial_range[1], num_points, dtype=np.float32) #генерируем num_points точек в интервале 
        z=tf.reshape(xs,[-1,1])
        return self.test(z, branch)

spatial_range = [-1, 1]
num_hidden = 40
batch_size = 200 #количество точек
num_iters = 100000
lr_rate = 1e-3
N = 41

VarPDE = deepRitz(spatial_range, num_hidden, batch_size, num_iters, lr_rate)
tic=time.time()
VarPDE.train()
toc=time.time()
elapsed=toc-tic
round(elapsed, 3)
varloss_list = VarPDE.loss_history
np.save('varloss_list.npy', varloss_list)

varloss_list = VarPDE.loss_history
np.save('varloss_list.npy', varloss_list)

u=VarPDE.numtest(N)
x = np.linspace(spatial_range[0], spatial_range[1], N, dtype=np.float32) #генерируем n точек в интервале [0,1]
plt.plot(x,u)
plt.title('Решение DNN')
plt.show()

x = np.linspace(spatial_range[0], spatial_range[1], N , dtype=np.float32) #генерируем n точек в интервале [0,1]
plt.plot(x, u, "r-")
x = np.linspace(spatial_range[0], 0, 41 , dtype=np.float32)
plt.plot(x, 1/2*x*(x+1),"g--")
x = np.linspace(0, spatial_range[1], 41 , dtype=np.float32) 
plt.plot(x, -1/2*x*(x-1),"g--")
plt.legend(['NN', 'u'])
# plt.title('lr_rate = 1e-2')
plt.show()

plt.plot(varloss_list, 'b')
plt.title('num_hidden = 40, batch_size = 10000')
plt.show()

ux=VarPDE.numtest(N, 2)
x=np.linspace(spatial_range[0],spatial_range[1], N)
plt.plot(x,ux,"r-")
x = np.linspace(spatial_range[0], 0, 41 , dtype=np.float32)
plt.plot(x, x+1/2,"g--")
x = np.linspace(0, spatial_range[1], 41 , dtype=np.float32)
plt.plot(x, -x+1/2,"g--")
plt.legend(['NN', 'u\''])
plt.show()
x=np.linspace(spatial_range[0],spatial_range[1], N)

def exact2(x):
  if x < spatial_range[0]:
    return 1/2*x*(x+1)
  elif spatial_range[0] <= x < 0:
    return 1/2*x*(x+1)
  elif 0 <= x <= spatial_range[1]:
    return -1/2*x*(x-1)
  elif x > spatial_range[1]:
    return -1/2*x*(x-1)


def exact_diff2(x):
  if x < spatial_range[0]:
    return x + 1/2
  elif spatial_range[0] <= x < 0:
    return x + 1/2
  elif 0 <= x <= spatial_range[1]:
    return -x + 1/2
  elif x > spatial_range[1]:
    return -x + 1/2

def simpson(f, a, b):
    val = (b - a) / 6 * (f(a) + 4 * f((a + b) / 2) + f(b))
    return val

def generalSimpson(f, a, b, eps):
  M = 3
  sum1 = simpson(f, a, b)
  flag = True
  while flag:
    sum2 = 0
    x = np.linspace(a, b, M)
    for i in range(1, M):
       sum2 += simpson(f, x[i-1], x[i])
    flag = abs(sum1 - sum2) > 15*eps
    sum1 = sum2
    M = (M - 1) * 2 + 1
  return sum2

def count_integral_error(exact, exact_diff, a, step, approx):
    diff = 0.0
    ddiff = 0.0
    v = a
    for i in range(len(approx) - 1):
        def _f(x):
          val = exact(x) - (approx[i] * (v + step - x) + approx[i + 1] * (x - v)) / step
          return val * val

        def _f1(x):
          val = exact_diff(x) - (approx[i + 1] - approx[i]) / step
          return val * val

        diff += generalSimpson(_f, v, v + step, 1e-8)
        ddiff += generalSimpson(_f1, v, v + step, 1e-8)
        v += step
    return math.sqrt(diff), math.sqrt(ddiff)

step = (spatial_range[1] - spatial_range[0]) / (N-1)
e1, e2 = count_integral_error(exact2, exact_diff2, spatial_range[0], step, u)
print(e1, e2)

def count_norm(exact, exact_diff, a, b):
  def _f(x):
    val = exact(x)
    return val * val

  def _f1(x):
    val = exact_diff(x)
    return val * val

  diff = generalSimpson(_f, a, b, 1e-8)
  ddiff = generalSimpson(_f1, a, b, 1e-8)
  return math.sqrt(diff), math.sqrt(ddiff)

norm_u, norm_u_diff = count_norm(exact2, exact_diff2, spatial_range[0], spatial_range[1])
print(norm_u, norm_u_diff)

round(e1 / norm_u * 100, 2)

round(e2 / norm_u_diff * 100, 2)

y = [1000, 5000, 10000, 20000, 100000]
# plt.plot(y, [11.02, 7.57, 8.72, 9.44])
plt.plot(y, [11.29, 13.0, 5.98, 6.78, 4.08])
# plt.plot(y, [21.08, 7.05, 6.79, 9.33])
plt.legend(['Ошибка для производных при 10 нейронах', 'Ошибка для производных при 20 нейронах','Ошибка для производных при 40 нейронах'])
plt.title('Зависимость ошибки от числа эпох, N=41')
plt.show()

y = [1000, 5000, 10000, 20000, 100000]
# plt.plot(y, [11.33, 10.05, 6.2, 6.04])
plt.plot(y, [11.42, 4.89, 9.08, 4.61, 5.68])
# plt.plot(y, [12.53, 7.22, 5.86, 6.12])
plt.legend(['Ошибка для производных при 10 нейронах', 'Ошибка для производных при 20 нейронах','Ошибка для производных при 40 нейронах'])
plt.title('Зависимость ошибки от числа эпох, N=81')
plt.show()

ModuleNotFoundError: No module named 'tensorflow'